In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()
os.chdir('drive')
os.chdir('MyDrive')
os.chdir('Coursera Project')

Mounted at /content/drive


In [ ]:
# Importing Necessary Libraries
import pandas as pd

In [ ]:
# Loading Transformed Data
Company_names = ['Amazon', 'Apple', 'Google', 'Meta', 'Netflix']
company_dict = {}
for i in Company_names:
  company_dict[i]=pd.read_csv(i+'_df.csv').set_index('Date')

In [ ]:
# Importing Necessary Libraries
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Setting up the batch size
batch_size = 64

In [ ]:
# Function to get Train, Test and Validation data
def construct_feature_target(df_in, f_len=3, batch_size = 64):
  df = df_in
  df_in = df_in.to_numpy()
  nbatch = len(df_in)//batch_size
  lendf = nbatch*batch_size
  X = df_in[:,:-1]
  y = df_in[:,-1:]
  X = np.reshape(X,(X.shape[0],X.shape[1],1))
  y = np.reshape(y,(y.shape[0],y.shape[1]))
  print(X.shape,y.shape)
  return df.index[0:int(0.8*nbatch)*batch_size], X[0:int(0.8*nbatch)*batch_size],y[0:int(0.8*nbatch)*batch_size], df.index[int(0.8*nbatch)*batch_size:int(0.9*nbatch)*batch_size],X[int(0.8*nbatch)*batch_size:int(0.9*nbatch)*batch_size],y[int(0.8*nbatch)*batch_size:int(0.9*nbatch)*batch_size], df.index[int(0.9*nbatch)*batch_size:int(1*nbatch)*batch_size], X[int(0.9*nbatch)*batch_size:int(1*nbatch)*batch_size],y[int(0.9*nbatch)*batch_size:int(1*nbatch)*batch_size], sc

In [ ]:
# Gettting Train, Test and Validation data and forming a dictionary for each company
df_train = {}
X_train = {}
y_train = {}
df_val = {}
X_val = {}
y_val = {}
df_test = {}
X_test = {}
y_test = {}
sc = {}
for i in Company_names:
  df_train[i], X_train[i], y_train[i], df_val[i], X_val[i], y_val[i], df_test[i],  X_test[i], y_test[i], sc[i]= construct_feature_target(company_dict[i], f_len=(company_dict[i].shape[1]-1), batch_size=batch_size)

(6464, 10, 1) (6464, 1)
(10624, 10, 1) (10624, 1)
(4608, 10, 1) (4608, 1)
(2688, 10, 1) (2688, 1)
(5184, 10, 1) (5184, 1)


In [ ]:
# Print shapes of Train, Test and Validation data for each company
for i in Company_names:
  print(X_train[i].shape, y_train[i].shape,  X_val[i].shape, y_val[i].shape, X_test[i].shape, y_test[i].shape)

(5120, 10, 1) (5120, 1) (640, 10, 1) (640, 1) (704, 10, 1) (704, 1)
(8448, 10, 1) (8448, 1) (1088, 10, 1) (1088, 1) (1088, 10, 1) (1088, 1)
(3648, 10, 1) (3648, 1) (448, 10, 1) (448, 1) (512, 10, 1) (512, 1)
(2112, 10, 1) (2112, 1) (256, 10, 1) (256, 1) (320, 10, 1) (320, 1)
(4096, 10, 1) (4096, 1) (512, 10, 1) (512, 1) (576, 10, 1) (576, 1)


In [ ]:
# Importing Keras Library
from tensorflow.keras.layers import Dense, Flatten 
from tensorflow.keras.layers import Input, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Setting up the architecture for the Stateful LSTM model
input = Input(batch_shape=(batch_size, (company_dict[i].shape[1]-1), 1))
lstm1 = LSTM(10, stateful=True, return_sequences=False)(input)
output = Dense(units=1)(lstm1)

In [ ]:
# Building a Stateful LSTM model for each company with adam optimizer and mse loss
regressor = {}
for i in Company_names:
  regressor[i] = Model(inputs=input, outputs=output)
  regressor[i].compile(optimizer='adam', loss='mse', metrics=['mean_absolute_error']) 
  regressor[i].summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(64, 10, 1)]             0         
                                                                 
 lstm (LSTM)                 (64, 10)                  480       
                                                                 
 dense (Dense)               (64, 1)                   11        
                                                                 
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(64, 10, 1)]             0         
                                                                 
 lstm (LSTM)                 (64, 10)         

In [ ]:
# Training the Stateful LSTM model with reset states on each epoch
callback = EarlyStopping(monitor='loss', patience=3)
for i in Company_names:
  for j in range(200):
    print(i,j)
    regressor[i].fit(x=X_train[i], y=y_train[i], validation_data=(X_val[i],y_val[i]), callbacks=[callback], shuffle=False, epochs=1, batch_size=batch_size)
    regressor[i].reset_states()


Amazon 0
80/80 [==============================] - 8s 10ms/step - loss: 1.9297e-04 - mean_absolute_error: 0.0096 - val_loss: 0.0157 - val_mean_absolute_error: 0.1163
Amazon 1
80/80 [==============================] - 0s 5ms/step - loss: 0.0016 - mean_absolute_error: 0.0281 - val_loss: 0.0135 - val_mean_absolute_error: 0.1096
Amazon 2
80/80 [==============================] - 0s 5ms/step - loss: 5.8272e-04 - mean_absolute_error: 0.0153 - val_loss: 0.0015 - val_mean_absolute_error: 0.0317
Amazon 3
80/80 [==============================] - 0s 5ms/step - loss: 4.4835e-05 - mean_absolute_error: 0.0047 - val_loss: 7.9502e-04 - val_mean_absolute_error: 0.0212
Amazon 4
80/80 [==============================] - 0s 5ms/step - loss: 1.2619e-05 - mean_absolute_error: 0.0024 - val_loss: 0.0011 - val_mean_absolute_error: 0.0263
Amazon 5
80/80 [==============================] - 0s 5ms/step - loss: 1.7299e-05 - mean_absolute_error: 0.0029 - val_loss: 0.0010 - val_mean_absolute_error: 0.0245
Amazon 6
80/80 

In [ ]:
# Getting the prdeictions of Stateful LSTM model on Test data
preds = {}
for i in Company_names:
  preds[i] = regressor[i].predict(X_test[i], batch_size = 64)

9/9 [==============================] - 0s 3ms/step


In [ ]:
# Loading Necessray Libraries
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Printing MAE, MSE and R2 score of the model on test data
for i in Company_names:
  print("--------------------------------------------------------------------------------------")
  print(f'Mean Absolute Error for {i} on test data is {mean_absolute_error(y_test[i],preds[i])}')
  print(f'Mean Squared Error for {i} on test data is {mean_squared_error(y_test[i],preds[i])}')
  print(f'R2 Score for {i} on test data is {r2_score(y_test[i],preds[i])}')
  print("--------------------------------------------------------------------------------------")

--------------------------------------------------------------------------------------
Mean Absolute Error for Amazon on test data is 0.028808977827228174
Mean Squared Error for Amazon on test data is 0.0011941236019233103
R2 Score for Amazon on test data is 0.9440528751903559
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
Mean Absolute Error for Apple on test data is 0.018273688483579992
Mean Squared Error for Apple on test data is 0.0006271887769028299
R2 Score for Apple on test data is 0.9891856971437744
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
Mean Absolute Error for Google on test data is 0.028545014421175734
Mean Squared Error for Google on test data is 0.0012383090729348816
R2 Score for Google on test data is 0.918395749629

In [ ]:
# Printing MAE, MSE and R2 score of the model on train data
for i in Company_names:
  print("--------------------------------------------------------------------------------------")
  print(f'Mean Absolute Error for {i} on train data is {mean_absolute_error(y_train[i],regressor[i].predict(X_train[i], batch_size = 64))}')
  print(f'Mean Squared Error for {i} on train data is {mean_squared_error(y_train[i],regressor[i].predict(X_train[i], batch_size = 64))}')
  print(f'R2 Score for {i} on train data is {r2_score(y_train[i],regressor[i].predict(X_train[i], batch_size = 64))}')
  print("--------------------------------------------------------------------------------------")

--------------------------------------------------------------------------------------
80/80 [==============================] - 0s 2ms/step
Mean Absolute Error for Amazon on train data is 0.008398736697318604
80/80 [==============================] - 0s 2ms/step
Mean Squared Error for Amazon on train data is 7.265286298075577e-05
80/80 [==============================] - 0s 2ms/step
R2 Score for Amazon on train data is 0.9821665568396568
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
132/132 [==============================] - 0s 2ms/step
Mean Absolute Error for Apple on train data is 0.008802600831871128
132/132 [==============================] - 0s 2ms/step
Mean Squared Error for Apple on train data is 7.802553154533575e-05
132/132 [==============================] - 0s 2ms/step
R2 Score for Apple on train data is 0.9141969806018986
------------------------------

In [ ]:
# Saving the final LSTM model for each company
for i in Company_names:
  regressor[i].save(i+'_trained_model.h5')

In [ ]:
load = {}
for i in Company_names:
  load[i]=load_model(i+'_trained_model.h5')

In [ ]:
for i in Company_names:
  print("--------------------------------------------------------------------------------------")
  print(f'Mean Absolute Error for {i} on train data is {mean_absolute_error(y_train[i],load[i].predict(X_train[i], batch_size = 64))}')
  print(f'Mean Squared Error for {i} on train data is {mean_squared_error(y_train[i],load[i].predict(X_train[i], batch_size = 64))}')
  print(f'R2 Score for {i} on train data is {r2_score(y_train[i],load[i].predict(X_train[i], batch_size = 64))}')
  print("--------------------------------------------------------------------------------------")

--------------------------------------------------------------------------------------
80/80 [==============================] - 1s 2ms/step
Mean Absolute Error for Amazon on train data is 0.00839490207520685
80/80 [==============================] - 0s 2ms/step
Mean Squared Error for Amazon on train data is 7.265286298075577e-05
80/80 [==============================] - 0s 2ms/step
R2 Score for Amazon on train data is 0.9821665568396568
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
132/132 [==============================] - 1s 2ms/step
Mean Absolute Error for Apple on train data is 0.00880147447394541
132/132 [==============================] - 0s 2ms/step
Mean Squared Error for Apple on train data is 7.802553154533575e-05
132/132 [==============================] - 0s 2ms/step
R2 Score for Apple on train data is 0.9141969806018986
--------------------------------

In [ ]:
! pip list yfinance

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.15.1
astropy                       5.2.1
astunparse                    1.6.3
atomicwrites                  1.4.1
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
bokeh                         2.4.3
branca                        0.6.0
bs4                           0.0.1
CacheControl                  0.12.11
cachetools                    5.3.0
catalogue                     2.0.8
certifi                       2022.12.7
